In [18]:
#Подключение небходимых библиотек
import qiskit, qiskit_aer
import qiskit.visualization
import qiskit_ibm_runtime
import random
import time
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager


In [19]:
#service = qiskit_ibm_runtime.QiskitRuntimeService(channel ='ibm_quantum', token='17efa62aa9873b5a041acbd771297e673778205cc443bc748e869e84330c331fc92ce25cd9f5b87f1432106dd7983eedac0deedc237296b7ed815d6200f31b5a')

In [20]:
qubits_num = 6 #количество необходимых кубитов
secret_length = 256 #длина секрета
secret = []
for i in range(secret_length): #генерация случайного секрета
    bit = random.randint(0,1)
    secret.append(bit)

In [21]:
#функция генерации начального состояния дилера
def generate_bell():
    qc = qiskit.QuantumCircuit(qubits_num, qubits_num) #указываем  
    #количество квантовых и классических регистров

    qc.x(0) #генерируем состояние B10
    qc.h(0) 
    qc.cx(0,1) 

    qc.h(2) #генерируем состояние B00
    qc.cx(2,3)

    qc.h(4) #генерируем состояние B11
    qc.x(5)
    qc.z(4)
    qc.z(5)
    qc.cx(4,5)

    return qc

In [22]:
#функция кодирования секрета
def secret_encode_bell(secret):
    schemes = [] #массив схем, содержащих весь секрет
    for i in range(0, len(secret),2):
        qc = generate_bell() #генерируем состояние Белла
        #для кодирования 00 ничего не делаем
        if secret[i] == 0 and secret[i+1] == 1: #кодируем 01
            qc.x(0)
        elif secret[i] == 1 and secret[i+1] == 0: #кодируем 10
            qc.z(0)
        elif secret[i] == 1 and secret[i+1] == 1: #кодируем 11
            qc.z(0)
            qc.x(0)
        schemes.append(qc)
    return schemes

In [23]:
ts1 = time.time()
#применяем функцию кодирования
encoded_secret = secret_encode_bell(secret)

In [24]:
#функция проведения измерений в базисе Белла
def bell_measurment(scheme):

    scheme.cx(0,4) #измерение участника P1
    scheme.h(0)
    scheme.measure(0,0)
    scheme.measure(4,4)

    scheme.cx(1,3) #измерение участника P2
    scheme.h(1)
    scheme.measure(1,1)
    scheme.measure(3,3)

    scheme.cx(2,5) #измерение участника P3
    scheme.h(2)
    scheme.measure(2,2)
    scheme.measure(5,5)

    return scheme

In [25]:
#функция для нахождения соответствий результатов парам битов
def results_to_bits():
    secret = [0,0,0,1,1,0,1,1] #кодируем все возможные пары битов
    results = [] #массив результатов измерений
    schemes = secret_encode_bell(secret) #генерируем изначальное состояние дилера 
    #и кодируем секрет
    for i in range(4):
        schemes[i] = bell_measurment(schemes[i]) #для каждой из схем проводим 
        #измерения
        backend = qiskit_aer.AerSimulator() #подключение симулятора
        
        circuit = qiskit.transpile(schemes[i], backend) #транспиляция схемы

        job = backend.run(circuit, shots=256) #многократно запускаем схему для 
        #получения всех возможных результатов измерений участников
            
        result = job.result() #считываем результаты измерений
        counts=result.get_counts()
        results.append(list(counts.keys()))
    return results

results = results_to_bits() #применяем функцию


In [26]:
#функция восстановления битов секрета по известной информации
def bit_recovering_bell(measures):
    measure = ''.join(map(str, measures)) #преобразуем результаты измерений из 
    #массива в строку
    result_00 = results[0] #результаты измерений участников, соответствующих битам 
    #00 и т.д.
    result_01 = results[1]
    result_10 = results[2]
    result_11 = results[3]
    if measure in result_00: #если полученный результат есть в 
        #каком-либо из массивов, то возвращаем соответствующую битовую пару
        return [0,0]
    elif measure in result_01:
        return [0,1]
    elif measure in result_10:
        return [1,0]
    elif measure in result_11:
        return [1,1]



In [27]:
#функция восстановления секрета
def secret_decode_bell(schemes):
    recovered_secret = [] #восстанавливаемый секрет

    for scheme in schemes:
        measures = []
        scheme = bell_measurment(scheme) #проводим измерения
        
        backend = qiskit_aer.AerSimulator() #подключение симулятора
        
        circuit = qiskit.transpile(scheme, backend) #транспиляция схемы

        job = backend.run(circuit, shots=1) #запускаем схему по одному разу для 
        #каждой пары битов
        
        result = job.result() #считываем результаты измерений
        counts=result.get_counts()
        
        measures = list(map(int, list(counts.keys())[0])) #преобразуем полученные 
        #результаты в массив битов
        
        recovered_bits = bit_recovering_bell(measures) #восстанавливаем отдельную 
        #битовую пару
        recovered_secret.append(recovered_bits) #добавляем биты к восстанавливаемому 
        #секрету
    recovered_secret = [x for y in recovered_secret for x in y]
    return recovered_secret


In [28]:
recovered_secret = secret_decode_bell(encoded_secret)
ts2 = time.time()
errors = 0 #количество ошибок
for i in range(secret_length): #подсчитываем количество ошибок
    if secret[i] != recovered_secret[i]:
        errors += 1
print(errors)
print(ts2-ts1)

0
6.180049180984497


In [29]:
total_qubits_num = 30 #количество кубит квантового процессора
qubits_num = 6 #количество необходимых кубитов для передачи двух битов секрета
secret_length = total_qubits_num//3 #длина секрета
secret = []
for i in range(secret_length): #генерация случайного секрета
    bit = random.randint(0,1)
    secret.append(bit)

In [30]:
#функция генерации начального состояния дилера на квантовом процессоре
def generate_bell_real():
    qc = qiskit.QuantumCircuit(total_qubits_num, total_qubits_num) #указываем  
    #количество квантовых и классических регистров
    for i in range(0, total_qubits_num - qubits_num + 1, qubits_num):
        qc.x(i) #генерируем состояние B10
        qc.h(i) 
        qc.cx(i,i+1) 

        qc.h(i+2) #генерируем состояние B00
        qc.cx(i+2,i+3)

        qc.h(i+4) #генерируем состояние B11
        qc.x(i+5)
        qc.z(i+4)
        qc.z(i+5)
        qc.cx(i+4,i+5)

    return qc

In [31]:
#функция кодирования секрета на квантовом процессоре
def secret_encode_bell_real(secret):
    qc = generate_bell_real()
    for i in range(0, len(secret),2):
        #для кодирования 00 ничего не делаем
        if secret[i] == 0 and secret[i+1] == 1: #кодируем 01
            qc.x(i*3)
        elif secret[i] == 1 and secret[i+1] == 0: #кодируем 10
            qc.z(i*3)
        elif secret[i] == 1 and secret[i+1] == 1: #кодируем 11
            qc.z(i*3)
            qc.x(i*3)
    return qc

#применяем функцию кодирования
encoded_secret_real = secret_encode_bell_real(secret)

In [32]:
#функция проведения измерений в базисе Белла для квантового процессора
def bell_measurment_real(scheme):
    for i in range(0,total_qubits_num//6):
        scheme.cx(i*6,i*6+4) #измерение участника P1
        scheme.h(i*6)
        scheme.measure(i*6,i*6)
        scheme.measure(i*6+4,i*6+4)

        scheme.cx(i*6+1,i*6+3) #измерение участника P2
        scheme.h(i*6+1)
        scheme.measure(i*6+1,i*6+1)
        scheme.measure(i*6+3,i*6+3)

        scheme.cx(i*6+2,i*6+5) #измерение участника P3
        scheme.h(i*6+2)
        scheme.measure(i*6+2,i*6+2)
        scheme.measure(i*6+5,i*6+5)

    return scheme

In [33]:
#функция восстановления секрета для квантового процессора
def secret_decode_bell_real(scheme):
    recovered_secret = [] #восстанавливаемый секрет
    measures = []
    scheme = bell_measurment_real(scheme) #проводим измерения
    
    backend = qiskit_aer.AerSimulator() #подключение симулятора
    #backend = service.backend(name="ibm_brisbane") #вместо симулятора выбираем  
    #квантовое устройство, в данном случае ibm_brisbane
        
    circuit = qiskit.transpile(scheme, backend) #транспиляция схемы

    job = backend.run(circuit, shots=1) #запускаем схему на квантовом процессоре 
    #один раз
        
    result = job.result() #считываем результаты измерений
    counts=result.get_counts()
    measures = list(map(int, list(counts.keys())[0])) #преобразуем полученные 
        #результаты в массив битов
     

    for i in range(len(measures)//6):
        
        bell_measure = measures[i*6:(i+1)*6] #тройка результатов измерений

        recovered_bits = bit_recovering_bell(bell_measure)

        recovered_secret.append(recovered_bits) #добавляем бит к восстанавливаемому 
        #секрету
    recovered_secret.reverse()
    recovered_secret = [x for y in recovered_secret for x in y]
    
    return recovered_secret

In [34]:
recovered_secret = secret_decode_bell_real(encoded_secret_real) #применяем функцию 
#декодирования секрета
errors = 0 #количество ошибок
for i in range(secret_length): #подсчитываем количество ошибок
    if secret[i] != recovered_secret[i]:
        errors += 1

qber = errors/secret_length*100 #QBER - величина, показывающая процент ошибок
print("QBER =", qber,"%")

[1, 0, 1, 0, 1, 1, 1, 0, 0, 1]
[1, 0, 1, 0, 1, 1, 1, 0, 0, 1]
QBER = 0.0 %


In [35]:
print("00:", sorted(results[0]))
print("01:", sorted(results[1]))
print("10:", sorted(results[2]))
print("11:", sorted(results[3]))
str1 = ""
for i in range(0,len(measurment_output),6):
    str1+=str(measurment_output[i])
    str1+=str(measurment_output[i+1])
    str1+=str(measurment_output[i+2])
    str1+=str(measurment_output[i+3])
    str1+=str(measurment_output[i+4])
    str1+=str(measurment_output[i+5])
    str1+=" "
print(str1)
recovered_secret.reverse()
str1 = ""
for i in range(0,len(recovered_secret),2):
    str1+=str(recovered_secret[i])
    str1+=str(recovered_secret[i+1])
    str1+= " "

print(str1)
secret.reverse()
str1 = ""
for i in range(0,len(secret),2):
    str1+=str(secret[i])
    str1+=str(secret[i+1])
    str1+= " "

print(str1)


00: ['001000', '001011', '001101', '001110', '010000', '010011', '010101', '010110', '100000', '100011', '100101', '100110', '111000', '111011', '111101', '111110']
01: ['000000', '000011', '000101', '000110', '011000', '011011', '011101', '011110', '101000', '101011', '101101', '101110', '110000', '110011', '110101', '110110']
10: ['001001', '001010', '001100', '001111', '010001', '010010', '010100', '010111', '100001', '100010', '100100', '100111', '111001', '111010', '111100', '111111']
11: ['000001', '000010', '000100', '000111', '011001', '011010', '011100', '011111', '101001', '101010', '101100', '101111', '110001', '110010', '110100', '110111']


NameError: name 'measurment_output' is not defined

In [327]:
#qiskit.visualization.plot_histogram(counts)
#backend = service.backend(name="ibm_brisbane")
#scheme.draw(output='mpl')